In [1]:
import numpy as np
import gymnasium as gym
import torch

import os 
import sys

sys.path.append(os.path.abspath('../..'))

from environment.env import POMDPDeformedGridworld

In [2]:
OBSERVATION_TYPE = 'cardinal' # 'cardinal' or  'single'
BELIEF_UPDATE = 'particlefilters' # 'discrete', 'variational' or 'particlefilters'
DISCRETIZATION = 2000 # 
DEBUG = False

# Load MDP solution

In [3]:

run = "PPO_continous_" + "enh53x0u"

from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3 import PPO
from environment.env import Grid
from utils.checkpoints import find_last_checkpoint

env = Grid(
    shear_range=(-.2, .2),
    stretch_range=(.4,1),
    render_mode="human"
)

last_checkpoint = find_last_checkpoint(f"../../agents/pretrained/MDP/{run}")
model = PPO.load(f"../../agents/pretrained/MDP/{run}/{last_checkpoint}", env=env)


env.close()

/home/flaccagora/.miniconda3/envs/robogym/lib/python3.11/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


# Load Obs Model

In [4]:
def load_obs_model(obs_type):
    from observation_model.obs_model import singleNN, cardinalNN

    if obs_type == 'single':
        obs_model = singleNN()
        obs_model.load_state_dict(torch.load("obs_model_4.pth", weights_only=True,map_location=torch.device('cpu')))
    elif obs_type == 'cardinal':
        obs_model = cardinalNN()
        obs_model.load_state_dict(torch.load("obs_model_cardinal_4.pth", weights_only=True))
    else:
        raise ValueError("Observation type not recognized")
    
    return obs_model
obs_model = load_obs_model(OBSERVATION_TYPE)

# Pomdp agent

In [5]:
import torch
import numpy as np
from collections import OrderedDict
from stable_baselines3 import PPO


class POMDPAgent():
    
    def __init__(self,mdp_agent:PPO, pomdp_env: POMDPDeformedGridworld, discretization=10, update='discrete', obs_model=None, debug=False):
        assert isinstance(pomdp_env, POMDPDeformedGridworld), f'Invalid environment type {type(pomdp_env)}'
        self.pomdp_env = pomdp_env
        self.mdp_agent = mdp_agent
        self.update = update

        if update == 'discrete' or update == 'discrete_exact':
            stretch = np.linspace(.4, 1, discretization)
            shear = np.linspace(-.2,.2, discretization)
            xa,ya,yb,xb = np.meshgrid(stretch, shear,shear,stretch) # , shear, shear
            positions = np.column_stack([xa.ravel(),ya.ravel(),yb.ravel(),xb.ravel()]),
            positions = torch.tensor(np.array(positions), dtype=torch.float32)
            self.belief_points = positions.squeeze()
            self.belief_values = torch.ones(self.belief_points.shape[0], dtype=torch.float32, requires_grad=False) / len(positions)
            
            self.original_def = env.transformation_matrix[0][0], env.transformation_matrix[1][1]

        if update == 'discrete': 
            assert obs_model is not None, f'Need an observation model for discrete belief update, given {obs_model}'
            self.obs_model = obs_model
            self.belief_update = self.discrete_belief_update
            print('Discrete belief update with observation model - SHEAR allowed')
        elif update == 'discrete_exact':
            self.belief_update = self.exact_belief_update
            raise NotImplementedError('Exact belief update not implemented here')
        elif update == 'variational':
            from utils.belief import BetaVariationalBayesianInference
            assert obs_model is not None, f'Need an observation model for variational belief update, given {obs_model}'
            self.VI = BetaVariationalBayesianInference(obs_model, input_dim=2, latent_dim=4, debug=debug)
            self.obs_model = obs_model

            self.belief_update = self.variational_belief_update
            self.X_history = [self.pomdp_env.get_state()['pos']]
            self.y_history = [self.pomdp_env.get_state()['obs']]
        elif update == 'particlefilters':
            from utils.belief import BayesianParticleFilter
            self.obs_model = obs_model
            self.n_particles = discretization
            self.PF = BayesianParticleFilter(f = obs_model, n_particles=self.n_particles, theta_dim=4)
            self.PF.initialize_particles()
            self.belief_update = self.particle_filter_belief_update
            
            self.X_history = [self.pomdp_env.get_state()['pos']]
            self.y_history = [self.pomdp_env.get_state()['obs']]

        else:
            raise ValueError('Invalid belief update method')
        
        self.debug = debug
        if self.debug:
            print(f'Using {update} belief update method')
    
    def predict(self, s, deterministic=True):
        
        self.belief_update(s)
        pos = s['pos']
        if self.update == 'discrete_exact' or self.update == 'discrete':
            # theta = self.belief_points[self.belief_values.argmax()] # QMDP
            theta = self.belief_points[torch.multinomial(self.belief_values, 1).item()] # Thompson sampling
        elif self.update == 'variational':
            theta = self.VI.sample_latent(1).squeeze().clone().detach().numpy() # variational Thompson sampling
        elif self.update == 'particlefilters':
            mean, var = self.PF.estimate_posterior()
            theta = torch.distributions.Normal(torch.tensor(mean), torch.tensor(var).sqrt()+1e-6).sample().squeeze()
            # theta = torch.tensor(mean, dtype=torch.float32)

        s = OrderedDict({'pos': pos, 'theta': theta})
        action = self.mdp_agent.predict(s, deterministic=deterministic)

        self.on_precidt_callback()
        return action
    
    def discrete_belief_update(self, pomdp_state):
        """discrete belief update"""
        pos = pomdp_state['pos']
        obs = pomdp_state['obs']

        batch_pos = pos.repeat(len(self.belief_points), 1)

        with torch.no_grad():
            predictions = self.obs_model(batch_pos,self.belief_points)

        likelihood = torch.exp(torch.distributions.Bernoulli(predictions).log_prob(obs))
        if len(likelihood.shape) == 2:
            likelihood = likelihood.sum(dim=1)

        tmp = likelihood.squeeze() * self.belief_values
        self.belief_values = tmp  / tmp.sum()

    def exact_belief_update(self, pomdp_state):
        """discrete belief update"""
        obs = pomdp_state['obs']
        pos = pomdp_state['pos']

        def f():
            likelihood = []
            for x in self.belief_points:
                try:
                    self.pomdp_env.set_deformation([x[0], x[1]],[0,0]) # stretch, shear format
                    likelihood.append(torch.all(torch.tensor(self.pomdp_env.observe(list(pos))) == obs))
                except:
                    raise ValueError('Invalid belief point x', x)
            self.pomdp_env.set_deformation(self.original_def, [0,0])
            return torch.tensor(likelihood, dtype=torch.float32)

        
        likelihood = f()
        self.belief_values =  likelihood * self.belief_values
        self.belief_values = self.belief_values / self.belief_values.sum()

    def variational_belief_update(self, pomdp_state):
        self.X_history.append(pomdp_state['pos'])
        self.y_history.append(pomdp_state['obs'])

        # X = posizione dell'agente (x,y)
        X = torch.stack(self.X_history)

        # ossevrazioni dell'agente negli stati pos=(x,y)
        y = torch.stack(self.y_history)

        # Create and fit the model
        self.VI.fit(X, y, n_epochs=10, lr=0.05)

    def particle_filter_belief_update(self, pomdp_state):
        self.X_history.append(pomdp_state['pos'])
        self.y_history.append(pomdp_state['obs'])

        # X = posizione dell'agente (x,y)
        X = torch.stack(self.X_history[-1:])

        # ossevrazioni dell'agente negli stati pos=(x,y)
        y = torch.stack(self.y_history[-1:])

        # X, y = pomdp_state['pos'].unsqueeze(0), pomdp_state['obs'].unsqueeze(0)

        # Create and fit the model
        self.PF.update(X, y)

    def on_precidt_callback(self):
        if self.debug:
            self.print_stats()
        
    def print_stats(self):
        if self.update == 'discrete':
            # print(f'Belief shape: {self.belief_values.shape}')
            # print(f'Belief points shape: {self.belief_points.shape}')
            # print(f'Belief max: {self.belief_points[self.belief_values.argmax()]}')
            # print(f'Belief sum: {self.belief_values.sum()}')
            # print(f'Belief entropy: {torch.distributions.Categorical(probs=self.belief_values).entropy()}')
            # print("\n")
            self.entropy = torch.distributions.Categorical(probs=self.belief_values).entropy()
        elif self.update == 'variational':
            # print(f'Variational inference: {self.VI.entropy()}')
            # print(self.VI.get_posterior_params())
            # print("\n")
            self.entropy = self.VI.entropy()
        elif self.update == 'particlefilters':
            # print(f'Particle filter: {self.PF.estimate_posterior()[1]}')
            # print("\n")
            self.entropy = None# self.PF.entropy()
        
    def reset(self):
        self.X_history = [self.pomdp_env.get_state()['pos']]
        self.y_history = [self.pomdp_env.get_state()['obs']]
        self.entropy = None

        if self.update == 'discrete':
            self.belief_values = torch.ones(self.belief_points.shape[0], dtype=torch.float32, requires_grad=False) / len(self.belief_points)
        elif self.update == 'variational':
            del self.VI
            from utils.belief import BetaVariationalBayesianInference
            self.VI = BetaVariationalBayesianInference(self.obs_model, input_dim=2, latent_dim=4, debug=self.debug)
        elif self.update == 'particlefilters':
            del self.PF
            from utils.belief import BayesianParticleFilter
            self.PF = BayesianParticleFilter(f = self.obs_model, n_particles=self.n_particles, theta_dim=4)
            self.PF.initialize_particles()

In [15]:
pomdp_env = POMDPDeformedGridworld(obs_type=OBSERVATION_TYPE)
pomdp_env.reset()

agent = POMDPAgent(model, pomdp_env, update=BELIEF_UPDATE, obs_model=obs_model,discretization=DISCRETIZATION, debug=DEBUG)

while True:
    try:
        action, _  = agent.predict(pomdp_env.get_state(), deterministic=False)
        _, _ , terminated, truncated, _ = pomdp_env.step(action)
        pomdp_env.render()
        if terminated or truncated:
            print('Terminated' if terminated else 'Truncated')
            print(pomdp_env.transformation_matrix)
            break
    except:
        print('Terminated' if terminated else 'Truncated')

        break
pomdp_env.close()   

Terminated
[[0.7250188149317318, 0.1756523169150862], [-0.17704976369063757, 0.5876180343994203]]


# MDP bound

In [7]:
from tqdm import trange 
def eval_agent_mdp(agent,env,num_episodes):
    """Returns
        - episode_transition: list of list of tuples (s,a,r,s',done), t[i] is the ith episode
        - beliefs: list of beliefs at each time step 
    """
    transitions = []
    for i in trange(num_episodes):
        s, _ = env.reset()

        totalReward = 0.0
        done = False
        steps = 0

        ep_transitions = []

        while not done:

            best_action, _ = agent.predict(s,deterministic=True)
            next_state, reward, terminated, truncated, info = env.step(best_action)
            totalReward += reward            

            done = terminated or truncated
            s = next_state
            steps += 1

            ep_transitions.append((s, best_action, reward, next_state, terminated, truncated))
    
        transitions.append(ep_transitions)

    env.close()

    return transitions

env = Grid(
    render_mode="rgb_array"
)

transitions = eval_agent_mdp(model,env,1000)

100%|██████████| 1000/1000 [00:26<00:00, 38.14it/s]


In [8]:
# chek how many times the agent has reached the goal
reached_goal = 0
for ep in transitions:
    if ep[-1][-2] == True:
        reached_goal += 1
print("Target reached", reached_goal, "out of", len(transitions), "episodes")    

# check the mean reward
mean_reward = 0
for ep in transitions:
    mean_reward += sum([t[2] for t in ep])
mean_reward /= len(transitions)
print("Mean episode Reward: ", mean_reward)

# check the mean number of steps
mean_steps = 0
for ep in transitions:
    mean_steps += len(ep)
mean_steps /= len(transitions)
print("Mean number of steps: ", mean_steps)

Target reached 992 out of 1000 episodes
Mean episode Reward:  -20.6195
Mean number of steps:  30.148


# EVALUATION

In [16]:
from tqdm import trange

def eval_agent_pomdp(agent:POMDPAgent,env: POMDPDeformedGridworld,num_episodes):
    """Returns
        - episode_transition: list of list of tuples (s,a,r,s',done), t[i] is the ith episode
        - beliefs: list of beliefs at each time step 
    """

    assert agent.debug, 'Agent must be in debug mode to evaluate'

    transitions = []
    entropy = []

    for i in trange(num_episodes):

        agent.reset()
        s, _ = env.reset()

        totalReward = 0.0
        done = False
        steps = 0
        episode_transitions = []
        episode_entropy = []
        
        while not done:

            best_action, _ = agent.predict(s, deterministic=True)

            next_state, reward, terminated, truncated, info = env.step(best_action)
            
            done = terminated or truncated
            s = next_state

            steps += 1
            totalReward += reward
            episode_transitions.append((s, best_action, reward, next_state, terminated, truncated))
            # episode_entropy.append(agent.entropy.item())

        transitions.append(episode_transitions)
        # entropy.append(episode_entropy)

    env.close()

    return transitions, entropy


In [17]:
OBSERVATION_TYPE = 'cardinal' # 'single' or 'cardinal' 
RENDER_MODE = 'rgb_array' # 'rgb_array' or 'human'
BELIEF_UPDATE = 'particlefilters' # 'variational' 'discrete' or 'particlefilters'
DISCRETIZATION = 7000
DEBUG = True

obs_model = load_obs_model(OBSERVATION_TYPE)

In [22]:
pomdp_env = POMDPDeformedGridworld(obs_type=OBSERVATION_TYPE, render_mode=RENDER_MODE)
agent = POMDPAgent(model, pomdp_env, update=BELIEF_UPDATE, obs_model=obs_model,discretization=DISCRETIZATION, debug=DEBUG)
transitions, entropy = eval_agent_pomdp(agent,pomdp_env, 100)

Using particlefilters belief update method


100%|██████████| 100/100 [00:35<00:00,  2.84it/s]


In [23]:
def explain_transitions(transitions):
    # chek how many times the agent has reached the goal
    reached_goal = 0
    for ep in transitions:
        if ep[-1][-2] == True:
            reached_goal += 1
    print("Target reached", reached_goal, "out of", len(transitions), "episodes")    

    # check the mean reward
    mean_reward = 0
    for ep in transitions:
        mean_reward += sum([t[2] for t in ep])
    mean_reward /= len(transitions)
    print("Mean episode Reward: ", mean_reward)

    # check the mean number of steps
    mean_steps = 0
    for ep in transitions:
        mean_steps += len(ep)
    mean_steps /= len(transitions)
    print("Mean number of steps: ", mean_steps)

explain_transitions(transitions)

Target reached 96 out of 100 episodes
Mean episode Reward:  -45.405
Mean number of steps:  58.74


In [ ]:
def plot_entropy(entropy):
    import matplotlib.pyplot as plt

    plt.figure(figsize=(10,6))
    for i in range(len(entropy)):
        plt.plot(entropy[i])

    plt.xlabel('Time step')
    plt.ylabel('Entropy')
    plt.title('Entropy over time')

plot_entropy(entropy)

In [24]:
reached_goal = 0
truncated_index = []
for i, ep in enumerate(transitions):
    if ep[-1][-2] == True:
        reached_goal += 1
    else:
        truncated_index.append(i)
print("Target reached", reached_goal, "out of", len(transitions), "episodes")    

# check the mean reward
mean_reward = 0
for i, ep in enumerate(transitions):
    if i in truncated_index:
        continue
    mean_reward += sum([t[2] for t in ep])
mean_reward /= len(transitions)
print("Mean episode Reward: ", mean_reward)

# check the mean number of steps
mean_steps = 0
for i, ep in enumerate(transitions):
    if i in truncated_index:
        continue
    mean_steps += len(ep)
mean_steps /= len(transitions)
print("Mean number of steps: ", mean_steps)


Target reached 96 out of 100 episodes
Mean episode Reward:  -20.865
Mean number of steps:  38.7


# Discrete vs VI

In [ ]:
OBSERVATION_TYPE = 'single' # OR 'cardinal'
RENDER_MODE = 'human' # OR 'rgb_array'
BELIEF_UPDATE = 'discrete' # OR 'variational'
DISCRETIZATION = 20
DEBUG = True

obs_model = load_obs_model(OBSERVATION_TYPE)

pomdp_env = POMDPDeformedGridworld(obs_type=OBSERVATION_TYPE, render_mode=RENDER_MODE)
agent = POMDPAgent(model, pomdp_env, update=BELIEF_UPDATE, obs_model=obs_model,discretization=DISCRETIZATION, debug=DEBUG)


In [ ]:
pomdp_env.reset()

stretch, shear = [pomdp_env.transformation_matrix[0][0], pomdp_env.transformation_matrix[1][1]],[pomdp_env.transformation_matrix[0][1], pomdp_env.transformation_matrix[1][0]]
starting_pos = pomdp_env.get_state()['pos']

print("Starting position: ", starting_pos)
print("Deformation: ", pomdp_env.transformation_matrix)

In [168]:
while True:
    try:
        action, _  = agent.predict(pomdp_env.get_state(), deterministic=False)
        _, _ , terminated, truncated, _ = pomdp_env.step(action)
        pomdp_env.render()
        if terminated or truncated:
            break
    except:
        break
pomdp_env.close()   

In [ ]:
OBSERVATION_TYPE = 'single' # OR 'cardinal'
RENDER_MODE = 'human' # OR 'rgb_array'
BELIEF_UPDATE = 'variational' # OR 'variational'
DISCRETIZATION = 20
DEBUG = True

obs_model = load_obs_model(OBSERVATION_TYPE)

pomdp_env = POMDPDeformedGridworld(obs_type=OBSERVATION_TYPE, render_mode=RENDER_MODE)
agent = POMDPAgent(model, pomdp_env, update=BELIEF_UPDATE, obs_model=obs_model,discretization=DISCRETIZATION, debug=DEBUG)


In [170]:
pomdp_env.set_deformation(stretch, shear)
pomdp_env.set_position(starting_pos.tolist())

In [ ]:
while True:
    try:
        action, _  = agent.predict(pomdp_env.get_state(), deterministic=False)
        _, _ , terminated, truncated, _ = pomdp_env.step(action)
        pomdp_env.render()
        if terminated or truncated:
            break
    except:
        break
pomdp_env.close()   